# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのスターターノートブックであり、特に「Rigging」というフレームワークを使用して、LLM（大規模言語モデル）が「20の質問」ゲームをプレイできるようにするために作成されています。

### 問題の内容
- コンペティションでは、参加者が言語モデルを使用して、限られた質問を通じて指定された単語を当てるゲームを行います。目指すべきは、最小限の質問で答えに到達することであり、そのためには効果的な質問を生成し、的確な回答を引き出すことが求められます。
- Notebook内では、`llama3`モデルを`phi3`モデルに切り替えて性能を改善し、LLMと非LLMエージェントを組み合わせた戦略を取っています。Riggingフレームワークを使用することで、異なるモデル間のインタラクションを容易にし、期待される出力を確認しながらLLMにクエリを送信できます。

### 使用されている手法とライブラリ
- **Rigging**: Pydantic XMLに基づいた軽量のLLMインタラクションフレームワークで、LLMの生産パイプラインを管理します。質問の生成や回答の取得を柔軟に行うことができます。
- **vLLM**: モデルをローカルサービスとしてホストするために使用され、データの取得やモデルのトレーニング環境を整える役割を果たします。
- 追加のライブラリとして、`huggingface_hub`や`pandas`が使用され、モデルの重みのダウンロードやデータ管理を効率化しています。

### ノートブックのワークフロー
1. **初期設定**: Hugging FaceとKaggleのトークンの取得、依存パッケージのインストールが行われる。
2. **モデルの準備**: `snapshot_download`を通じてHugging Faceからモデルの重みをダウンロードします。
3. **vLLMサーバーの起動**: ローカル環境でモデルをホストし、LLMが質問と回答を行えるようにします。
4. **質問者と回答者のエージェント作成**: Pythonクラスを利用して、質問を生成し、回答を処理するロジックが整備されています。
5. **キーワードデータフレームの作成**: 公開テーマについてのキーワードが格納され、どの単語に対して質問するかを決定するためのベースになります。
6. **最終提出物の作成**: 実行環境の準備が整い、`main.py`を含むZIPファイルが作成されます。

このノートブックは、Kaggleのコンペティション用にLLMを効果的に活用できる基盤を提供しており、Riggingを利用することで、競技者が簡単にLLMのインタラクションを構築し、テストし、調整できるように設計されています。

---


# 用語概説 
以下は、Jupyter Notebookに関連する専門用語の解説です。特に、初心者がつまずきやすいマイナーな用語や実務を経験していないと馴染みがない用語に焦点を当てました。

1. **Rigging**:
   - 軽量なLLMインタラクションフレームワークであり、Pydantic XMLに基づいて構築されています。特に、バックエンドのLLMモデルを簡単に入れ替えたり、LLMクエリパイプラインを設計するために使用されます。LLM（大規模言語モデル）とのインタラクションを効率化することを目的としています。

2. **Pydantic**:
   - Pythonのデータバリデーションと設定管理を行うためのライブラリです。特に、データモデルの定義を簡単に行える機能を提供し、型ヒントを用いたバリデーションが可能です。

3. **vLLM**:
   - モデルをローカルでホストするためのライブラリであり、特にLLMを効率的にサーバーとして運用するためのツールです。特に非同期的な処理をサポートしており、高速な応答が求められる環境に適しています。

4. **Activation-aware Weight Quantization**:
   - モデルの軽量化手法の一つで、モデルのパラメータを量子化（サイズを小さくする）する際に、活性化のビット幅も考慮する技術です。この手法は、モデルの推論時のパフォーマンスを改善するために使用されることがあります。

5. **XML (eXtensible Markup Language)**:
   - データの構造を記述するためのマークアップ言語で、特にデータの交換や保存に用いられます。このノートブックでは、LLMの入力や出力フォーマットとして使用されています。

6. **ChatPipeline**:
   - Riggingにおいて、LLMとの対話の流れを定義するためのモデルであり、特定のチャットの操作や応答管理を担います。LLMとのインタラクションを効率化するための一連のステップを含んでいます。

7. **Subprocess**:
   - Pythonで別のプログラムやスクリプトを実行するためのモジュールであり、特に外部コマンドを効率的に呼び出す際に使用されます。ここでは、vLLMサーバーをバックグラウンドで起動するために用いられています。

8. **Parsing**:
   - 入力されたデータを特定の形式や構造に従って解釈したり変換したりするプロセスです。LLMの出力を適切な形式で抽出するために使用されます。

9. **Field Validator**:
   - Pydanticでモデルのフィールド値が要件を満たしているかを確認するためのメソッドで、データの整合性を保証する役割を担います。特にそのモデル内で定義されます。

10. **Observation**:
    - ゲームの進行状況や状態を保持するためのデータ構造で、質問や回答、推測をトラックするのに使用されます。ゲームのロジックを管理するために重要な役割を果たします。

これらの用語は、Notebookでの具体的な実例やコードに関連しているため、より理解を深めるための助けとなるでしょう。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# LLM 20 Questions Starter with Rigging [Phi3]

## This notebook is a revised version of an earlier notebook. We have replaced the `llama3` model with the `phi3` model as requested by @bhanupm. You can find the original notebook [here](https://www.kaggle.com/code/robikscube/phi3-intro-to-rigging-for-llm-20-questions/).

In my early testing the llama3 version performs better, but this might be improved with better prompting.

This starter notebook shows how the python package rigging can be used to create a baseline submission for the competition. This setup uses the `phi3` quantized model using vLLM.

## Update **June 10, 2024**
- Updated code to work with rigging 2.0
- Including non-llm question asking agent that leverages the known keywords **note this won't work well on the private leaderboard**. Answer agent uses LLM via rigging.

## What is Rigging?

Rigging is a lightweight LLM interaction framework built on Pydantic XML. The goal is to make leveraging LLMs in production pipelines as simple and effictive as possible. Rigging is perfectly fit for the 20 questions tasks as it can:
1. Easily handle swapping out different backend LLM models.
2. Design LLM querying pipelines that check for expected outputs and retry until successful.
3. Modern python with type hints, async support, pydantic validation, serialization, etc.

Star the repo here: https://github.com/dreadnode/rigging
Read the documentation here: https://rigging.dreadnode.io/

Rigging is built and maintained by [dreadnode](https://www.dreadnode.io/) where we use it daily for our work.

An example rigging pipeline might look like this:
```{python}
chat = rg.get_generator('gpt-4o') \
    .chat(f"Provide me the names of all the countries in South America that start with the letter A {Answer.xml_tags()} tags.") \
    .until_parsed_as(Answer) \
    .run() 
```

Generators can be created seemlessly with most major LLM apis, so long as you have api keys saved as env variables.
```
export OPENAI_API_KEY=...
export TOGETHER_API_KEY=...
export TOGETHERAI_API_KEY=...
export MISTRAL_API_KEY=...
export ANTHROPIC_API_KEY=...
```

For this competition we must run our model locally, luckily rigging has support to run models using transformers on the back end.

# Setup

Below is some of the setup for this notebook. Where we will:
- Load secret tokens for huggingface and kaggle (optional)
- Install required packages
- Create a helper utility script for testing our vLLM server

This notebooks uses some hidden tokens using kaggle's secrets. This is optional and not required to run the code.

</div>
<div class="column-right">

# 日本語訳

# LLM 20 Questions スターター with Rigging [Phi3]

## このノートブックは、以前のノートブックの改訂版です。`llama3`モデルを@bhanupmのリクエストに応じて`phi3`モデルに置き換えました。元のノートブックは[こちら](https://www.kaggle.com/code/robikscube/phi3-intro-to-rigging-for-llm-20-questions/)で見つけることができます。

初期のテストでは、llama3バージョンがより良いパフォーマンスを発揮しましたが、より良いプロンプトによって改善される可能性があります。

このスターターノートブックでは、コンペティションのためのベースライン提出を作成する方法を示します。このセットアップは、vLLMを使用した`phi3`量子化モデルを利用しています。

## 更新 **2024年6月10日**
- rigging 2.0に対応するようにコードを更新しました
- Known keywordsを活用する非LLM質問エージェントを含めました **これはプライベートリーダーボードではうまく機能しません**。回答エージェントはriggingを介してLLMを使用します。

## Riggingとは？

Riggingは、Pydantic XMLに基づいて構築された軽量なLLMインタラクションフレームワークです。目的は、LLMを生産パイプラインでできるだけ簡単かつ効果的に活用できるようにすることです。Riggingは20の質問タスクに完全に適しており、以下のことが可能です：
1. 異なるバックエンドLLMモデルを簡単に入れ替えることができます。
2. 期待される出力をチェックし、成功するまで繰り返すLLMクエリパイプラインを設計できます。
3. 現代のPythonで、型ヒント、非同期サポート、pydanticバリデーション、シリアル化などをサポートしています。

リポジトリをこちらでスターしておいてください: https://github.com/dreadnode/rigging
ドキュメントをこちらでお読みください: https://rigging.dreadnode.io/

Riggingは[dreadnode](https://www.dreadnode.io/)によって構築され、維持されています。私たちは日常的にそれを使用しています。

例として、以下のようなriggingパイプラインが考えられます：
```{python}
chat = rg.get_generator('gpt-4o') \
    .chat(f"Provide me the names of all the countries in South America that start with the letter A {Answer.xml_tags()} tags.") \
    .until_parsed_as(Answer) \
    .run() 
```

生成器は、大多数の主要なLLM APIとシームレスに作成できます。APIキーが環境変数として保存されている限り、問題ありません。
```
export OPENAI_API_KEY=...
export TOGETHER_API_KEY=...
export TOGETHERAI_API_KEY=...
export MISTRAL_API_KEY=...
export ANTHROPIC_API_KEY=...
```

このコンペティションではモデルをローカルで実行する必要があるため、幸運にもriggingはトランスフォーマーをバックエンドに使用してモデルを実行するサポートがあります。

# セットアップ

以下は、このノートブックのいくつかのセットアップです。ここでは以下のことを行います：
- Hugging FaceとKaggleのための秘密トークンをロードします（オプション）
- 必要なパッケージをインストールします
- vLLMサーバーをテストするためのヘルパーユーティリティスクリプトを作成します

このノートブックは、Kaggleの秘密を使用していくつかの隠されたトークンを利用します。これはオプションであり、コードを実行するためには必要ありません。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None
KAGGLE_KEY: str | None = None
KAGGLE_USERNAME: str | None = None
    
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    pass
```

</div>
<div class="column-right">

# 日本語訳

```python
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None
KAGGLE_KEY: str | None = None
KAGGLE_USERNAME: str | None = None
    
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    pass
```

</div>
</details>

In [ ]:
from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None
KAGGLE_KEY: str | None = None
KAGGLE_USERNAME: str | None = None
    
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    pass

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Pip install
We will install:
- [rigging](https://github.com/dreadnode/rigging) Used to created our LLM pipelines for the competition.
- [vLLM](https://github.com/vllm-project/vllm) For hosting our model locally as an independent service.

We also use [uv](https://github.com/astral-sh/uv) which allows us to install these packages much faster.

**Note:** We are installing these packages to the `/kaggle/tmp/lib` directory. We only do this for the purposes of the competition setup, where we will later need to include the files from this path in our submission zip. We also install the vllm dependencies to `/kaggle/tmp/srvlib`.

</div>
<div class="column-right">

# 日本語訳

## パッケージのインストール
私たちは以下をインストールします：
- [rigging](https://github.com/dreadnode/rigging) コンペティションのためのLLMパイプラインを作成するために使用します。
- [vLLM](https://github.com/vllm-project/vllm) モデルをローカルで独立したサービスとしてホストするために使用します。

また、[uv](https://github.com/astral-sh/uv)を使用しており、これによりこれらのパッケージをより迅速にインストールできます。

**注意:** これらのパッケージは`/kaggle/tmp/lib`ディレクトリにインストールされています。これはコンペティションのセットアップ目的のために行い、このパス内のファイルを後で提出用のZIPに含める必要があります。また、vllmの依存パッケージも`/kaggle/tmp/srvlib`にインストールします。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Dependencies (uv for speed)
!pip install uv==0.1.45

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==2.0.0 \
    kaggle

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm==0.4.2 \
    numpy==1.26.4
```

</div>
<div class="column-right">

# 日本語訳

```python
# 依存関係（速度向上のためのuv）
!pip install uv==0.1.45

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==2.0.0 \
    kaggle

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm==0.4.2 \
    numpy==1.26.4
```

</div>
</details>

In [ ]:
# 依存関係（速度向上のためのuv）
!pip install uv==0.1.45

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==2.0.0 \
    kaggle

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm==0.4.2 \
    numpy==1.26.4

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Download the LLM Locally

Because this competition requires us to submit our code with model weights, we will first download the model weights using `snapshot_download` from huggingface.

We are going to download the `solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ`. This is a Activation-aware Weight Quantization version of the model that is small enough to run in the competition requirements.

**Note**: When using rigging in a normal situation this step would not be necessary, but we are downloading the weights seperately so that we can include them in our submission zip for the competition.

</div>
<div class="column-right">

# 日本語訳

# LLMをローカルにダウンロードする

このコンペティションでは、モデルの重みを含むコードを提出する必要があるため、まず`snapshot_download`を使用してHugging Faceからモデルの重みをダウンロードします。

`solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ`をダウンロードします。これは、コンペティション要件に十分に小さいActivation-aware Weight Quantizationバージョンのモデルです。

**注意:** 通常の状況でriggingを使用する場合、このステップは必要ありませんが、コンペティション用に提出物に含めるために重みを別々にダウンロードしています。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Download the model

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)
g_model_path.mkdir(parents=True)

snapshot_download(
    repo_id="rhysjones/Phi-3-mini-mango-1-llamafied",
    ignore_patterns="original*",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=globals().get("HF_TOKEN", None)
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# モデルをダウンロードする

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)
g_model_path.mkdir(parents=True)

snapshot_download(
    repo_id="rhysjones/Phi-3-mini-mango-1-llamafied",
    ignore_patterns="original*",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=globals().get("HF_TOKEN", None)
)
```

</div>
</details>

In [ ]:
# モデルをダウンロードする

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)
g_model_path.mkdir(parents=True)

snapshot_download(
    repo_id="rhysjones/Phi-3-mini-mango-1-llamafied",
    ignore_patterns="original*",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=globals().get("HF_TOKEN", None)
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

We can see the model weights are stored in `/kaggle/tmp/model/`

</div>
<div class="column-right">

# 日本語訳

モデルの重みが`/kaggle/tmp/model/`に保存されているのが確認できます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!ls -l /kaggle/tmp/model
```

</div>
<div class="column-right">

# 日本語訳

```python
!ls -l /kaggle/tmp/model
```

</div>
</details>

In [ ]:
!ls -l /kaggle/tmp/model

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Helper Utilities File

These are helper functions we will use for starting our vLLM server.

</div>
<div class="column-right">

# 日本語訳

# ヘルパーユーティリティファイル

これらは、vLLMサーバーを起動するために使用するヘルパー関数です。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile util.py

# Helpers for starting the vLLM server

import subprocess
import os
import socket
import time

def check_port(port: int) -> bool:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                return True
    except socket.error:
        pass
    
    return False

def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60, debug: bool = False
) -> subprocess.Popen:
    
    if check_port(port):
        raise ValueError(f"Port {port} is already open")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},
        stdout=subprocess.DEVNULL if not debug else None,
        stderr=subprocess.DEVNULL if not debug else None,
    )
    
    start_time = time.time()
    while time.time() - start_time < timeout:
        if check_port(port):
            return popen
        time.sleep(1)
    
    popen.terminate()
    raise Exception(f"Process did not open port {port} within {timeout} seconds.")
```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile util.py

# vLLMサーバーを起動するためのヘルパー

import subprocess
import os
import socket
import time

def check_port(port: int) -> bool:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                return True
    except socket.error:
        pass
    
    return False

def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60, debug: bool = False
) -> subprocess.Popen:
    
    if check_port(port):
        raise ValueError(f"ポート {port} はすでに開いています")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},
        stdout=subprocess.DEVNULL if not debug else None,
        stderr=subprocess.DEVNULL if not debug else None,
    )
    
    start_time = time.time()
    while time.time() - start_time < timeout:
        if check_port(port):
            return popen
        time.sleep(1)
    
    popen.terminate()
    raise Exception(f"プロセスが {port} を {timeout} 秒以内に開きませんでした。")
```

</div>
</details>

In [ ]:
%%writefile util.py

# vLLMサーバーを起動するためのヘルパー

import subprocess
import os
import socket
import time

def check_port(port: int) -> bool:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                return True
    except socket.error:
        pass
    
    return False

def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60, debug: bool = False
) -> subprocess.Popen:
    
    if check_port(port):
        raise ValueError(f"ポート {port} はすでに開いています")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},
        stdout=subprocess.DEVNULL if not debug else None,
        stderr=subprocess.DEVNULL if not debug else None,
    )
    
    start_time = time.time()
    while time.time() - start_time < timeout:
        if check_port(port):
            return popen
        time.sleep(1)
    
    popen.terminate()
    raise Exception(f"プロセスが {port} を {timeout} 秒以内に開きませんでした。")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Starting up our vLLM server for testing

Our model will be hosted using a vLLM server. Below we will start up the notebook so we can understand how it works in the kaggle environment.

</div>
<div class="column-right">

# 日本語訳

# テストのためにvLLMサーバーを起動する

私たちのモデルは、vLLMサーバーを使用してホストされます。以下では、Kaggle環境での動作を理解するためにノートブックを起動します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# vLLM paths and settings.

import importlib
from pathlib import Path
import util

util = importlib.reload(util)

g_srvlib_path = Path("/kaggle/tmp/srvlib")
assert g_srvlib_path.exists()

g_model_path = Path("/kaggle/tmp/model")
assert g_model_path.exists()

g_vllm_port = 9999
g_vllm_model_name = "custom"
```

</div>
<div class="column-right">

# 日本語訳

```python
# vLLMのパスと設定。

import importlib
from pathlib import Path
import util

util = importlib.reload(util)

g_srvlib_path = Path("/kaggle/tmp/srvlib")
assert g_srvlib_path.exists()

g_model_path = Path("/kaggle/tmp/model")
assert g_model_path.exists()

g_vllm_port = 9999
g_vllm_model_name = "custom"
```

</div>
</details>

In [ ]:
# vLLMのパスと設定。

import importlib
from pathlib import Path
import util

util = importlib.reload(util)

g_srvlib_path = Path("/kaggle/tmp/srvlib")
assert g_srvlib_path.exists()

g_model_path = Path("/kaggle/tmp/model")
assert g_model_path.exists()

g_vllm_port = 9999
g_vllm_model_name = "custom"

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Run the vLLM server using subprocess
vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",
    "--enforce-eager",
    "--model", str(g_model_path),
    "--port", str(g_vllm_port),
    "--served-model-name", g_vllm_model_name,
    "--dtype=half"
],
    g_vllm_port,
    {"PYTHONPATH": str(g_srvlib_path)},
    debug=False
)

print("vLLM Started")
```

</div>
<div class="column-right">

# 日本語訳

```python
# サブプロセスを使用してvLLMサーバーを実行する
vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",
    "--enforce-eager",
    "--model", str(g_model_path),
    "--port", str(g_vllm_port),
    "--served-model-name", g_vllm_model_name,
    "--dtype=half"
],
    g_vllm_port,
    {"PYTHONPATH": str(g_srvlib_path)},
    debug=False
)

print("vLLMが起動しました")
```

</div>
</details>

In [ ]:
# サブプロセスを使用してvLLMサーバーを実行する
vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",
    "--enforce-eager",
    "--model", str(g_model_path),
    "--port", str(g_vllm_port),
    "--served-model-name", g_vllm_model_name,
    "--dtype=half"
],
    g_vllm_port,
    {"PYTHONPATH": str(g_srvlib_path)},
    debug=False
)

print("vLLMが起動しました")

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

We can see that the llama3 model is loaded onto the 1st Tesla T4 GPU.

</div>
<div class="column-right">

# 日本語訳

llama3モデルが最初のTesla T4 GPUにロードされているのが確認できます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!nvidia-smi
```

</div>
<div class="column-right">

# 日本語訳

```python
!nvidia-smi
```

</div>
</details>

In [ ]:
!nvidia-smi

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Validating the Model

Lets create our first rigging generator. In rigging the generators are the foundation for creating powerful LLM pipelines.

</div>
<div class="column-right">

# 日本語訳

## モデルの検証

最初のrigging生成器を作成します。riggingにおいて、生成器は強力なLLMパイプラインを作成する基本です。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Connect with Rigging

import sys
import logging

sys.path.insert(0, "/kaggle/tmp/lib")

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

import rigging as rg

generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llama requires some hand holding
)

answer = await generator.chat("Say Hello!").run()

print()
print('[Rigging Chat]')
print(type(answer), answer)

print()
print('[LLM Response Only]')
print(type(answer.last), answer.last)

print()
answer_string = answer.last.content
print('[LLM Response as a String]')
print(answer.last.content)
```

</div>
<div class="column-right">

# 日本語訳

```python
# Riggingとの接続

import sys
import logging

sys.path.insert(0, "/kaggle/tmp/lib")

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

import rigging as rg

generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

answer = await generator.chat("Say Hello!").run()

print()
print('[Rigging Chat]')
print(type(answer), answer)

print()
print('[LLMのレスポンスのみ]')
print(type(answer.last), answer.last)

print()
answer_string = answer.last.content
print('[LLMレスポンスを文字列として]')
print(answer.last.content)
```

</div>
</details>

In [ ]:
# Riggingとの接続

import sys
import logging

sys.path.insert(0, "/kaggle/tmp/lib")

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

import rigging as rg

generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

answer = await generator.chat("Say Hello!").run()

print()
print('[Rigging Chat]')
print(type(answer), answer)

print()
print('[LLMのレスポンスのみ]')
print(type(answer.last), answer.last)

print()
answer_string = answer.last.content
print('[LLMレスポンスを文字列として]')
print(answer.last.content)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Converting results to pandas dataframe

Using the `to_df()` method we can easily convert the chat history to a pandas dataframe.

</div>
<div class="column-right">

# 日本語訳

## 結果をpandasデータフレームに変換する

`to_df()`メソッドを使用することで、チャット履歴を簡単にpandasデータフレームに変換できます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
answer.to_df()
```

</div>
<div class="column-right">

# 日本語訳

```python
answer.to_df()
```

</div>
</details>

In [ ]:
answer.to_df()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Changing Model Parameters

Much like database connection strings, Rigging generators can be represented as strings which define what provider, model, API key, generation params, etc. should be used. They are formatted as follows:

```
<provider>!<model>,<**kwargs>
```

As an example, here we load the model with additional parameters:
- temperature=0.9
- max_tokens=512

You can read more about these in the docs here: https://rigging.dreadnode.io/topics/generators/#overload-generation-params

</div>
<div class="column-right">

# 日本語訳

## モデルパラメーターの変更

データベース接続文字列と同様に、Rigging生成器は使用するプロバイダー、モデル、APIキー、生成パラメータなどを定義する文字列として表現できます。それらは以下の形式にフォーマットされます：

```
<provider>!<model>,<**kwargs>
```

例えば、以下のように追加のパラメータを使用してモデルをロードします：
- temperature=0.9
- max_tokens=512

これらについての詳細は、ドキュメントのこちらをお読みください: https://rigging.dreadnode.io/topics/generators/#overload-generation-params


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=0.9,max_tokens=512," \
    "stop=<|eot_id|>" # Llama requires some hand holding,
)
```

</div>
<div class="column-right">

# 日本語訳

```python
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=0.9,max_tokens=512," \
    "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)
```

</div>
</details>

In [ ]:
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=0.9,max_tokens=512," \
    "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Alternatively we can set these parameters using the `rg.GenerateParams` class. This class allows you to set various model parameters:

```
rg.GenerateParams(
    *,
    temperature: float | None = None,
    max_tokens: int | None = None,
    top_k: int | None = None,
    top_p: float | None = None,
    stop: list[str] | None = None,
    presence_penalty: float | None = None,
    frequency_penalty: float | None = None,
    api_base: str | None = None,
    timeout: int | None = None,
    seed: int | None = None,
    extra: dict[str, typing.Any] = None,
)
```

https://rigging.dreadnode.io/api/generator/#rigging.generator.GenerateParams

</div>
<div class="column-right">

# 日本語訳

また、`rg.GenerateParams`クラスを使用してこれらのパラメータを設定することもできます。このクラスを使用すると、さまざまなモデルパラメータを設定できます：

```
rg.GenerateParams(
    *,
    temperature: float | None = None,
    max_tokens: int | None = None,
    top_k: int | None = None,
    top_p: float | None = None,
    stop: list[str] | None = None,
    presence_penalty: float | None = None,
    frequency_penalty: float | None = None,
    api_base: str | None = None,
    timeout: int | None = None,
    seed: int | None = None,
    extra: dict[str, typing.Any] = None,
)
```

https://rigging.dreadnode.io/api/generator/#rigging.generator.GenerateParams


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
rg_params = rg.GenerateParams(
    temperature = 0.9,
    max_tokens = 512,
)
base_chat = generator.chat(params=rg_params)
answer = await base_chat.fork('How is it going?').run()
print(answer.last.content)
```

</div>
<div class="column-right">

# 日本語訳

```python
rg_params = rg.GenerateParams(
    temperature = 0.9,
    max_tokens = 512,
)
base_chat = generator.chat(params=rg_params)
answer = await base_chat.fork('How is it going?').run()
print(answer.last.content)
```

</div>
</details>

In [ ]:
rg_params = rg.GenerateParams(
    temperature = 0.9,
    max_tokens = 512,
)
base_chat = generator.chat(params=rg_params)
answer = await base_chat.fork('How is it going?').run()
print(answer.last.content)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

Or parameters can be set within the chain using params.

</div>
<div class="column-right">

# 日本語訳

また、パラメータはチェーン内でparamsを使用して設定することもできます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
base_chat = generator.chat() # No params set
answer = await base_chat.fork('How is it going?') \
    .with_(temperature = 0.9, max_tokens = 512) \
    .run()
print(answer.last.content)
```

</div>
<div class="column-right">

# 日本語訳

```python
base_chat = generator.chat() # パラメータは設定されていません
answer = await base_chat.fork('How is it going?') \
    .with_(temperature = 0.9, max_tokens = 512) \
    .run()
print(answer.last.content)
```

</div>
</details>

In [ ]:
base_chat = generator.chat() # パラメータは設定されていません
answer = await base_chat.fork('How is it going?') \
    .with_(temperature = 0.9, max_tokens = 512) \
    .run()
print(answer.last.content)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Parsed outputs example

Next we will create a pipeline where we:
1. Create a rigging Model called `Answer`. This explains the expected output that we will parse from the model results.
    - We will add some validators to this that will ensure the output is either `yes` or `no`
    - This is fully customizable.
    - Here `validate_content` is ensuring that our response conforms to the expected output (lowercase and starts with "yes" or "no")
2. We can use the `Answer.xml_example()` in our prompt to let the LLM know how we expect the output to look.
3. Later on we will use `.until_parsed_as(Answer)` to ensure the LLM output is extracted as defined here.

**Note** `until_parsed_as()` can take a `max_rounds` parameter, which by default is 5.

</div>
<div class="column-right">

# 日本語訳

# パースされた出力の例

次に、以下の手順を実行するパイプラインを作成します：
1. `Answer`というriggingモデルを作成します。これにより、モデルの結果からパースする期待出力が説明されます。
    - ここで、出力が`yes`または`no`であることを確実にするために、いくつかのバリデーターを追加します。
    - これは完全にカスタマイズ可能です。
    - ここで`validate_content`は、私たちのレスポンスが期待される出力（小文字で始まり、"yes"または"no"で始まる）に準拠していることを確認します。
2. 私たちは、プロンプトに`Answer.xml_example()`を使用して、LLMに出力がどのように見えるべきかを知らせます。
3. 後で`.until_parsed_as(Answer)`を使用して、LLMの出力がここで定義されたように抽出されることを保証します。

**注意** `until_parsed_as()`は、デフォルトで5の`max_rounds`パラメータを取ることができます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import typing as t
from pydantic import field_validator

class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("Invalid answer, must be 'yes' or 'no'")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"

```

</div>
<div class="column-right">

# 日本語訳

```python
import typing as t
from pydantic import field_validator

class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です、'yes'または'no'である必要があります")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"
```

</div>
</details>

In [ ]:
import typing as t
from pydantic import field_validator

class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です、'yes'または'no'である必要があります")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Lets see what the xml example looks like for this we can use this in our prompt
Answer.xml_example()
```

</div>
<div class="column-right">

# 日本語訳

```python
# このxmlの例がプロンプト内でどのように表示されるかを見てみましょう
Answer.xml_example()
```

</div>
</details>

In [ ]:
# このxmlの例がプロンプト内でどのように表示されるかを見てみましょう
Answer.xml_example()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
#     "stop=<|eot_id|>" # Llama requires some hand holding,
)

keyword='Tom Hanks'
category='Famous Person'
last_question='Is it a famous person?'

prompt = f"""\
            The secret word for this game is "{keyword}" [{category}]

            You are currently answering a question about the word above.

            The next question is "{last_question}".

            Answer the yes/no question above and place it in the following format:
            {Answer.xml_example()}

            - Your response should be accurate given the keyword above
            - Always answer with "yes" or "no"

            What is the answer?
"""

chat = await (
    generator
    .chat(prompt)
    .until_parsed_as(Answer, max_rounds=50)
    .run()
)

print('=== Full Chat ===')
print(chat)

print()
print('=== LLM Response Only ===')
print(chat.last)

print()
print('=== Parsed Answer ===')
print(chat.last.parse(Answer).content)
```

</div>
<div class="column-right">

# 日本語訳

```python
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
#     "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

keyword='Tom Hanks'
category='有名な人'
last_question='それは有名な人ですか？'

prompt = f"""\
            このゲームの秘密の言葉は "{keyword}" です [{category}]

            現在、上記の言葉についての質問に答えています。

            次の質問は "{last_question}" です。

            上記のはい/いいえの質問に答え、以下のフォーマットにして下さい：
            {Answer.xml_example()}

            - あなたのレスポンスは、上記のキーワードに対して正確であるべきです
            - 常に "yes" または "no" で回答してください

            答えは何ですか？
"""

chat = await (
    generator
    .chat(prompt)
    .until_parsed_as(Answer, max_rounds=50)
    .run()
)

print('=== フルチャット ===')
print(chat)

print()
print('=== LLMのレスポンスのみ ===')
print(chat.last)

print()
print('=== パースされた回答 ===')
print(chat.last.parse(Answer).content)
```

</div>
</details>

In [ ]:
generator = rg.get_generator(
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
#     "stop=<|eot_id|>" # Llamaには少し手助けが必要です
)

keyword='Tom Hanks'
category='有名な人'
last_question='それは有名な人ですか？'

prompt = f"""\
            このゲームの秘密の言葉は "{keyword}" です [{category}]

            現在、上記の言葉についての質問に答えています。

            次の質問は "{last_question}" です。

            上記のはい/いいえの質問に答え、以下のフォーマットにして下さい：
            {Answer.xml_example()}

            - あなたのレスポンスは、上記のキーワードに対して正確であるべきです
            - 常に "yes" または "no" で回答してください

            答えは何ですか？
"""

chat = await (
    generator
    .chat(prompt)
    .until_parsed_as(Answer, max_rounds=50)
    .run()
)

print('=== フルチャット ===')
print(chat)

print()
print('=== LLMのレスポンスのみ ===')
print(chat.last)

print()
print('=== パースされた回答 ===')
print(chat.last.parse(Answer).content)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Create an example Questioner Chat Pipeline with Rigging

Next lets create the questioner pipeline that will attempt to help determine what the keyword might be.

First lets create a `Question` object which we will use to parse our output.

</div>
<div class="column-right">

# 日本語訳

# Riggingで質問者チャットパイプラインの例を作成する

次に、キーワードが何であるかを特定するために役立つ質問者パイプラインを作成します。

まず、出力をパースするために使用する`Question`オブジェクトを作成します。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from pydantic import StringConstraints  # noqa

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()
```

</div>
<div class="column-right">

# 日本語訳

```python
from pydantic import StringConstraints  # noqa

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**質問**").to_pretty_xml()
```

</div>
</details>

In [ ]:
from pydantic import StringConstraints  # noqa

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**質問**").to_pretty_xml()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
base =  generator.chat("""\
You are a talented player of the 20 questions game. You are accurate, focused, and
structured in your approach. You will create useful questions, make guesses, or answer
questions about a keyword.

""")


question_chat = await (base.fork(
    f"""\
    You are currently asking the next question.

    question and place it in the following format:
    {Question.xml_example()}

    - Your response should be a focused question which will gather the most information
    - Start general with your questions
    - Always try to bisect the remaining search space
    - Pay attention to previous questions and answers

    What is your next question?
    """
)
.until_parsed_as(Question, attempt_recovery=True)
.run()
)
```

</div>
<div class="column-right">

# 日本語訳

```python
base =  generator.chat("""\
あなたは20の質問ゲームの優れたプレイヤーです。あなたは正確で、集中しており、構造的なアプローチをとります。あなたは有用な質問を作成し、質問に対する回答を行ったり、キーワードに関する推測を行ったりします。

""")


question_chat = await (base.fork(
    f"""\
    あなたは現在次の質問をしています。

    質問を以下のフォーマットにして下さい：
    {Question.xml_example()}

    - あなたのレスポンスは、最も多くの情報を集める焦点を絞った質問であるべきです
    - 質問は一般的なものから始めてください
    - 残りの探索空間を二分するようにして下さい
    - 以前の質問と回答に注意してください

    あなたの次の質問は何ですか？
    """
)
.until_parsed_as(Question, attempt_recovery=True)
.run()
)
```

</div>
</details>

In [ ]:
base =  generator.chat("""\
あなたは20の質問ゲームの優れたプレイヤーです。あなたは正確で、集中しており、構造的なアプローチをとります。あなたは有用な質問を作成し、質問に対する回答を行ったり、キーワードに関する推測を行ったりします。

""")


question_chat = await (base.fork(
    f"""\
    あなたは現在次の質問をしています。

    質問を以下のフォーマットにして下さい：
    {Question.xml_example()}

    - あなたのレスポンスは、最も多くの情報を集める焦点を絞った質問であるべきです
    - 質問は一般的なものから始めてください
    - 残りの探索空間を二分するようにして下さい
    - 以前の質問と回答に注意してください

    あなたの次の質問は何ですか？
    """
)
.until_parsed_as(Question, attempt_recovery=True)
.run()
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Dataframe representation of the conversation
question_chat.to_df()
```

</div>
<div class="column-right">

# 日本語訳

```python
# 会話のデータフレーム表現
question_chat.to_df()
```

</div>
</details>

In [ ]:
# 会話のデータフレーム表現
question_chat.to_df()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

We now are confident that the LLM response contains the quesion and case parse the question like:

</div>
<div class="column-right">

# 日本語訳

私たちはLLMのレスポンスが質問を含んでいることを自信を持って確認し、以下のように質問を抽出することができます：


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
question = question_chat.last.parse(Question).content
print(question)
```

</div>
<div class="column-right">

# 日本語訳

```python
question = question_chat.last.parse(Question).content
print(question)
```

</div>
</details>

In [ ]:
question = question_chat.last.parse(Question).content
print(question)

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Create a keyword dataframe
** Note this only works because we know the possible keywords in the public set. This will not work on the final leaderboard**

</div>
<div class="column-right">

# 日本語訳

# キーワードデータフレームを作成する
** これは、公開されたセットの中での可能なキーワードを知っているためのみ機能します。最終的なリーダーボードでは機能しません**


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
```

</div>
<div class="column-right">

# 日本語訳

```python
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py
```

</div>
</details>

In [ ]:
!wget -O keywords_local.py https://raw.githubusercontent.com/Kaggle/kaggle-environments/master/kaggle_environments/envs/llm_20_questions/keywords.py

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!head keywords_local.py
```

</div>
<div class="column-right">

# 日本語訳

```python
!head keywords_local.py
```

</div>
</details>

In [ ]:
!head keywords_local.py

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import sys
import json
import pandas as pd
sys.path.append('./')
from keywords_local import KEYWORDS_JSON

def capitalize_first_word(text):
    if not text:
        return text
    return text[0].upper() + text[1:].lower()

def create_keyword_df(KEYWORDS_JSON):
    keywords_dict = json.loads(KEYWORDS_JSON)

    category_words_dict = {}
    all_words = []
    all_cat_words = []
    for d in keywords_dict:
        words = [w['keyword'] for w in d['words']]
        cat_word = [(d['category'], w['keyword']) for w in d['words']]
        category_words_dict[d['category']] = words
        all_words += words
        all_cat_words += cat_word

    keyword_df = pd.DataFrame(all_cat_words, columns=['category','keyword'])
    keyword_df['first_letter'] = keyword_df['keyword'].str[0]
    keyword_df['second_letter'] = keyword_df['keyword'].str[1]
    keyword_df.to_parquet('keywords.parquet')
    
create_keyword_df(KEYWORDS_JSON)
```

</div>
<div class="column-right">

# 日本語訳

```python
import sys
import json
import pandas as pd
sys.path.append('./')
from keywords_local import KEYWORDS_JSON

def capitalize_first_word(text):
    if not text:
        return text
    return text[0].upper() + text[1:].lower()

def create_keyword_df(KEYWORDS_JSON):
    keywords_dict = json.loads(KEYWORDS_JSON)

    category_words_dict = {}
    all_words = []
    all_cat_words = []
    for d in keywords_dict:
        words = [w['keyword'] for w in d['words']]
        cat_word = [(d['category'], w['keyword']) for w in d['words']]
        category_words_dict[d['category']] = words
        all_words += words
        all_cat_words += cat_word

    keyword_df = pd.DataFrame(all_cat_words, columns=['category','keyword'])
    keyword_df['first_letter'] = keyword_df['keyword'].str[0]
    keyword_df['second_letter'] = keyword_df['keyword'].str[1]
    keyword_df.to_parquet('keywords.parquet')
    
create_keyword_df(KEYWORDS_JSON)
```

</div>
</details>

In [ ]:
import sys
import json
import pandas as pd
sys.path.append('./')
from keywords_local import KEYWORDS_JSON

def capitalize_first_word(text):
    if not text:
        return text
    return text[0].upper() + text[1:].lower()

def create_keyword_df(KEYWORDS_JSON):
    keywords_dict = json.loads(KEYWORDS_JSON)

    category_words_dict = {}
    all_words = []
    all_cat_words = []
    for d in keywords_dict:
        words = [w['keyword'] for w in d['words']]
        cat_word = [(d['category'], w['keyword']) for w in d['words']]
        category_words_dict[d['category']] = words
        all_words += words
        all_cat_words += cat_word

    keyword_df = pd.DataFrame(all_cat_words, columns=['category','keyword'])
    keyword_df['first_letter'] = keyword_df['keyword'].str[0]
    keyword_df['second_letter'] = keyword_df['keyword'].str[1]
    keyword_df.to_parquet('keywords.parquet')
    
create_keyword_df(KEYWORDS_JSON)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
keywords_df = pd.read_parquet('keywords.parquet')
keywords_df.sample(10)
```

</div>
<div class="column-right">

# 日本語訳

```python
keywords_df = pd.read_parquet('keywords.parquet')
keywords_df.sample(10)
```

</div>
</details>

In [ ]:
keywords_df = pd.read_parquet('keywords.parquet')
keywords_df.sample(10)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
keywords_df['category'].value_counts()
```

</div>
<div class="column-right">

# 日本語訳

```python
keywords_df['category'].value_counts()
```

</div>
</details>

In [ ]:
keywords_df['category'].value_counts()

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Create `main.py` Script for Final Submission.

Our final submission will be a zipped directory with a `main` file. This file is below.

</div>
<div class="column-right">

# 日本語訳

# 最終提出用の`main.py`スクリプトを作成する。

私たちの最終提出物は、`main`ファイルを含むZIP圧縮ディレクトリになります。以下にそのファイルがあります。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile main.py

# Main agent file

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

import string
import numpy as np
import pandas as pd

# Path fixups

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"
    g_model_path = g_agent_path / "model"
    g_srvlib_path = g_agent_path / "srvlib"
else:
    g_agent_path = Path('/kaggle/working')
    
sys.path.insert(0, str(g_lib_path))

# Logging noise

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# Fixed imports

import util # noqa
import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# Constants

g_vllm_port = 9999
g_vllm_model_name = "custom"

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
)

# Types

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("Invalid answer, must be 'yes' or 'no'")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="question").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="thing/place").to_pretty_xml()


# Functions

async def ask(base: rg.ChatPipeline, observation: Observation) -> str:
    if observation.step == 0:
        # override first question until keyword bug is fixed.
        return "Are we playing 20 questions?"
    
    
    full_question = f"""\
                You are currently asking the next question.

                <game-history>
                {observation.get_history_as_xml(skip_guesses=True)}
                </game-history>

                Based on the history above, ask the next most useful yes/no
                question and place it in the following format:
                {Question.xml_example()}

                - Your response should be a focused question which will gather the most information
                - Start general with your questions
                - Always try to bisect the remaining search space
                - Pay attention to previous questions and answers

                What is your next question?
                """
    
    print(' ======Asking Question ======')
    print(full_question)
    
    
    try:
        chat = await (
             base.fork(full_question)
            .until_parsed_as(Question, attempt_recovery=True, max_rounds=20)
            .run()
        )
        return chat.last.parse(Question).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'Is it a person?'

async def answer(base: rg.ChatPipeline, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("Keyword wasn't provided to answerer", file=sys.stderr)
        return "yes" # override until keyword bug is fixed.
            
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            # Loop 5 times and take the most frequent response
            chat = await (
                base.fork(
#                     f"""\
#                         20 Questions game. Answer yes/no for this keyword: [{observation.keyword}]

#                             Question: {last_question}

#                             Rules:
#                             1. Only consider [{observation.keyword}]
#                             2. Check each letter for letter questions
#                             3. Answer only yes or no

#                             Format:
#                             <answer>yes</answer>
#                             OR
#                             <answer>no</answer>

#                             Your answer:
#                         """
                    f"""
                    Keyword: [{observation.keyword}]

                    Q: {last_question}

                    Answer yes or no in Format: <answer>yes</answer> OR <answer>no</answer>
                    """
                )
                .until_parsed_as(Answer, attempt_recovery=True, max_rounds=20)
                .run()
            )
            responses.append(chat.last.parse(Answer).content.strip('*'))
            
        print(f'Responses are {responses}')
        return pd.Series(responses).value_counts().index[0]
    except rg.error.MessagesExhaustedMaxRoundsError:
        print('%%%%%%%%%%%% Error so answering yes %%%%%%%%%%%% ')
        return 'yes'

async def guess(base: rg.ChatPipeline, observation: Observation) -> str:
    try:

        chat = await (
            base.fork(
                f"""\
                You are currently making an informed guess of the keyword.

                <game-history>
                {observation.get_history_as_xml()}
                </game-history>

                Based on the history above, produce a single next best guess
                for the keyword and place it in the following format:
                {Guess.xml_example()}

                - Avoid repeat guesses based on the history above
                - The guess should be a specific person, place, or thing

                What is your guess?
                """
            )
            .until_parsed_as(Guess, attempt_recovery=True, max_rounds=20)
            .run()
        )

        return chat.last.parse(Guess).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'france'
    
# vLLM and Generator

try:
    vllm = util.run_and_wait_for_port([
        "python", "-m",
        "vllm.entrypoints.openai.api_server",
        "--enforce-eager",
        "--model", str(g_model_path),
        "--port", str(g_vllm_port),
        "--served-model-name", g_vllm_model_name,
        "--dtype=half"
    ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

    print("vLLM Started")
except ValueError:
    print('vLLM Already Running')
    
    
generator = rg.get_generator(g_generator_id)

base =  generator.chat("""\
You are a talented player of the 20 questions game. You are accurate, focused, and
structured in your approach. You will create useful questions, make guesses, or answer
questions about a keyword.

""")

# Entrypoint
def format_first_letter_question(letters):
    if not letters:
        return "Does the keyword start with any letter?"
    
    if len(letters) == 1:
        return f"Does the keyword start with the letter '{letters[0]}'"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" or '{letters[-1]}'"
    
    return f"Does the keyword start with one of the letters {formatted_letters}?"

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

# Simple question asker
class SimpleQuestionerAgent():
    def __init__(self, keyword_df: pd.DataFrame):
        self.keyword_df = keyword_df
        self.keyword_df_init = keyword_df.copy()
        self.round = 0
        self.category_questions = [
            "Are we playing 20 questions?",
            "Is the keyword a thing that is not a location?",
            "Is the keyword a place?",
        ]
        self.found_category = False
        
    def filter_keywords(self, obs):
        print(self.keyword_df.shape)
        # Filter down keyword_df based on past answers
        for i, answer in enumerate(obs.answers):
            if obs.questions[i] in self.category_questions:
                if answer == 'yes':
                    if obs.questions[i] == "Is the keyword a thing that is not a location?":
                        self.found_category = 'things'
                    if obs.questions[i] == "Is the keyword a place?":
                        self.found_category = 'place'
                    fc = self.found_category
                    self.keyword_df = self.keyword_df.query('category == @fc').reset_index(drop=True)
    
            if obs.questions[i].startswith('Does the keyword start '):
                if self.keyword_df['first_letter'].nunique() <= 1:
                    break
                letter_question = obs.questions[i]
                letters = extract_letters_from_question(letter_question)
                self.keyword_df = self.keyword_df.reset_index(drop=True).copy()
                if obs.answers[i] == 'yes':
                    self.keyword_df = self.keyword_df.loc[
                        self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
                elif obs.answers[i] == 'no':
                    self.keyword_df = self.keyword_df.loc[
                        ~self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
        if len(self.keyword_df) == 0:
            # Reset
            self.keyword_df = self.keyword_df_init.copy()
            
    def get_letters(self, obs, max_letters=20):
        n_letters = self.keyword_df['first_letter'].nunique()
        sample_letters = self.keyword_df['first_letter'].drop_duplicates().sample(n_letters // 2).values.tolist()
        sample_letters = sample_letters[:max_letters]
        print('sample letters', n_letters, sample_letters)
        return sample_letters # ', '.join(sample_letters)
    
    def __call__(self, obs, *args):
        if len(self.keyword_df) == 0:
            # Reset
            self.keyword_df = self.keyword_df_init.copy()
        self.filter_keywords(obs)
        if obs.turnType == 'ask':
            self.round += 1
            if (self.round <= 3 and not self.found_category):
                response = self.category_questions[self.round - 1]
            else:
                sample_letters = self.get_letters(obs)
                if len(sample_letters) == 0:
                    n_sample = min(len(self.keyword_df), 10)
                    possible_keywords = ", ".join(self.keyword_df['keyword'].sample(n_sample).values.tolist())
                    response = f"Is the keyword one of the following? {possible_keywords}"
                else:
                    sample_letters_str = str(sample_letters).replace('[','').replace(']','')
#                     response = f'Does the keyword start with one of the following letters : {sample_letters_str} ?'
                    response = format_first_letter_question(sample_letters)
        elif obs.turnType == 'guess':
            response = self.keyword_df['keyword'].sample(1).values[0]
            # Remove the guessed word
            updated_df = self.keyword_df.loc[self.keyword_df['keyword'] != response].reset_index(drop=True).copy()
            if len(updated_df) >= 1:
                self.keyword_df = updated_df.copy()
            else:
                self.keyword_df = self.keyword_df_init.copy() # Reset the df
#         print(f'Round {self.round}')
#         print(f"{response=}")
#         print(f'keyword_df size {self.keyword_df.shape}')
        return response


keyword_df = pd.read_parquet(f'{g_agent_path}/keywords.parquet')
question_agent = None

async def observe(obs: t.Any) -> str:
    observation = Observation(**obs.__dict__)
    global question_agent
    if question_agent is None:
        question_agent = SimpleQuestionerAgent(keyword_df)

    try:
        match observation.turnType:
            case "ask":
#                 return await ask(base, observation)
                return question_agent(obs)
            case "answer":
                return await answer(base, observation)
            case "guess":
#                 return await guess(base, observation)
                return question_agent(obs)

            case _:
                raise ValueError("Unknown turn type")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    # Async gate when executing in their framework
    import asyncio
    return asyncio.run(observe(obs))

```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile main.py

# メインエージェントファイル

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

import string
import numpy as np
import pandas as pd

# パスの修正

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"
    g_model_path = g_agent_path / "model"
    g_srvlib_path = g_agent_path / "srvlib"
else:
    g_agent_path = Path('/kaggle/working')
    
sys.path.insert(0, str(g_lib_path))

# ロギングのノイズを無効にする

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# 固定されたインポート

import util # noqa
import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# 定数

g_vllm_port = 9999
g_vllm_model_name = "custom"

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
)

# タイプ

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です、'yes'または'no'である必要があります")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="質問").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="物/場所").to_pretty_xml()


# 関数

async def ask(base: rg.ChatPipeline, observation: Observation) -> str:
    if observation.step == 0:
        # キーワードバグが修正されるまで、最初の質問をオーバーライド
        return "私たちは20の質問をプレイしていますか？"
    
    
    full_question = f"""\
                あなたは現在次の質問をしています。

                <game-history>
                {observation.get_history_as_xml(skip_guesses=True)}
                </game-history>

                上記の履歴に基づき、次に最も有用なはい/いいえの質問をし、以下のフォーマットにしてください：
                {Question.xml_example()}

                - あなたのレスポンスは、最も多くの情報を集める焦点を絞った質問であるべきです
                - 質問は一般的なものから始めてください
                - 残りの探索空間を二分してください
                - 以前の質問と回答に注意してください

                あなたの次の質問は何ですか？
                """
    
    print(' ======質問中 ======')
    print(full_question)
    
    
    try:
        chat = await (
             base.fork(full_question)
            .until_parsed_as(Question, attempt_recovery=True, max_rounds=20)
            .run()
        )
        return chat.last.parse(Question).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return '人ですか？'

async def answer(base: rg.ChatPipeline, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("回答者にキーワードが提供されていません", file=sys.stderr)
        return "yes" # キーワードバグが修正されるまでオーバーライド
    
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            # 5回ループし、最も頻繁なレスポンスを取得
            chat = await (
                base.fork(
#                     f"""\
#                         20の質問ゲーム。次のキーワードに対してはい/いいえで回答してください：[ {observation.keyword} ]

#                             質問: {last_question}

#                             ルール:
#                             1.  [{observation.keyword}] のみを考慮してください
#                             2. 各文字の質問のために文字を確認してください
#                             3. はいまたはいいえのみで答えてください

#                             フォーマット:
#                             <answer>yes</answer>
#                             または
#                             <answer>no</answer>

#                             あなたの回答は：
#                         """
                    f"""
                    キーワード: [ {observation.keyword} ]

                    Q: {last_question}

                    フォーマット: <answer>yes</answer> または <answer>no</answer>ではいまたはいいえで答えてください
                    """
                )
                .until_parsed_as(Answer, attempt_recovery=True, max_rounds=20)
                .run()
            )
            responses.append(chat.last.parse(Answer).content.strip('*'))
            
        print(f'レスポンスは {responses}')
        return pd.Series(responses).value_counts().index[0]
    except rg.error.MessagesExhaustedMaxRoundsError:
        print('%%%%%%%%%%%% エラーが発生したため、yesを回答します %%%%%%%%%%%% ')
        return 'yes'

async def guess(base: rg.ChatPipeline, observation: Observation) -> str:
    try:

        chat = await (
            base.fork(
                f"""\
                あなたは現在キーワードの情報に基づく推測を行っています。

                <game-history>
                {observation.get_history_as_xml()}
                </game-history>

                上記の歴史に基づき、キーワードの次に最適な推測を1つ生成し、以下のフォーマットにしてください：
                {Guess.xml_example()}

                - 上記の歴史に基づいて重複した推測を避けてください
                - 推測は特定の人物、場所、または物である必要があります

                あなたの推測は何ですか？
                """
            )
            .until_parsed_as(Guess, attempt_recovery=True, max_rounds=20)
            .run()
        )

        return chat.last.parse(Guess).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'フランス'
    
# vLLMとGenerator

try:
    vllm = util.run_and_wait_for_port([
        "python", "-m",
        "vllm.entrypoints.openai.api_server",
        "--enforce-eager",
        "--model", str(g_model_path),
        "--port", str(g_vllm_port),
        "--served-model-name", g_vllm_model_name,
        "--dtype=half"
    ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

    print("vLLMが起動しました")
except ValueError:
    print('vLLMはすでに実行中です')
    
    
generator = rg.get_generator(g_generator_id)

base =  generator.chat("""\
あなたは20の質問ゲームの優れたプレイヤーです。あなたは正確で、集中しており、構造的なアプローチをとります。あなたは有用な質問を作成し、質問に対する回答を行ったり、キーワードに関する推測を行ったりします。

""")

# エントリーポイント
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは文字 '{letters[0]}' で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次のいずれかの文字で始まりますか: {formatted_letters}?"

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

# シンプルな質問者
class SimpleQuestionerAgent():
    def __init__(self, keyword_df: pd.DataFrame):
        self.keyword_df = keyword_df
        self.keyword_df_init = keyword_df.copy()
        self.round = 0
        self.category_questions = [
            "私たちは20の質問をプレイしていますか？",
            "キーワードは場所ではないものですか？",
            "キーワードは場所ですか？",
        ]
        self.found_category = False
        
    def filter_keywords(self, obs):
        print(self.keyword_df.shape)
        # 以前の回答に基づいてkeyword_dfをフィルタリング
        for i, answer in enumerate(obs.answers):
            if obs.questions[i] in self.category_questions:
                if answer == 'yes':
                    if obs.questions[i] == "キーワードは場所ではないものですか？":
                        self.found_category = 'things'
                    if obs.questions[i] == "キーワードは場所ですか？":
                        self.found_category = 'place'
                    fc = self.found_category
                    self.keyword_df = self.keyword_df.query('category == @fc').reset_index(drop=True)
    
            if obs.questions[i].startswith('キーワードは次の文字で始まりますか'):
                if self.keyword_df['first_letter'].nunique() <= 1:
                    break
                letter_question = obs.questions[i]
                letters = extract_letters_from_question(letter_question)
                self.keyword_df = self.keyword_df.reset_index(drop=True).copy()
                if obs.answers[i] == 'yes':
                    self.keyword_df = self.keyword_df.loc[
                        self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
                elif obs.answers[i] == 'no':
                    self.keyword_df = self.keyword_df.loc[
                        ~self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
            
    def get_letters(self, obs, max_letters=20):
        n_letters = self.keyword_df['first_letter'].nunique()
        sample_letters = self.keyword_df['first_letter'].drop_duplicates().sample(n_letters // 2).values.tolist()
        sample_letters = sample_letters[:max_letters]
        print('サンプル文字', n_letters, sample_letters)
        return sample_letters # ', '.join(sample_letters)
    
    def __call__(self, obs, *args):
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
        self.filter_keywords(obs)
        if obs.turnType == 'ask':
            self.round += 1
            if (self.round <= 3 and not self.found_category):
                response = self.category_questions[self.round - 1]
            else:
                sample_letters = self.get_letters(obs)
                if len(sample_letters) == 0:
                    n_sample = min(len(self.keyword_df), 10)
                    possible_keywords = ", ".join(self.keyword_df['keyword'].sample(n_sample).values.tolist())
                    response = f"キーワードは次のいずれかのものでしょうか？ {possible_keywords}"
                else:
                    sample_letters_str = str(sample_letters).replace('[','').replace(']','')
                    response = format_first_letter_question(sample_letters)
        elif obs.turnType == 'guess':
            response = self.keyword_df['keyword'].sample(1).values[0]
            # 推測された単語を取り除く
            updated_df = self.keyword_df.loc[self.keyword_df['keyword'] != response].reset_index(drop=True).copy()
            if len(updated_df) >= 1:
                self.keyword_df = updated_df.copy()
            else:
                self.keyword_df = self.keyword_df_init.copy() # dfをリセット
#         print(f'ラウンド {self.round}')
#         print(f"{response=}")
#         print(f'keyword_dfサイズ {self.keyword_df.shape}')
        return response


keyword_df = pd.read_parquet(f'{g_agent_path}/keywords.parquet')
question_agent = None

async def observe(obs: t.Any) -> str:
    observation = Observation(**obs.__dict__)
    global question_agent
    if question_agent is None:
        question_agent = SimpleQuestionerAgent(keyword_df)

    try:
        match observation.turnType:
            case "ask":
                return question_agent(obs)
            case "answer":
                return await answer(base, observation)
            case "guess":
                return question_agent(obs)

            case _:
                raise ValueError("不明なターンタイプ")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    # 彼らのフレームワークで実行するときの非同期ゲート
    import asyncio
    return asyncio.run(observe(obs))
```

</div>
</details>

In [ ]:
%%writefile main.py

# メインエージェントファイル

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

import string
import numpy as np
import pandas as pd

# パスの修正

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"
    g_model_path = g_agent_path / "model"
    g_srvlib_path = g_agent_path / "srvlib"
else:
    g_agent_path = Path('/kaggle/working')
    
sys.path.insert(0, str(g_lib_path))

# ロギングのノイズを無効にする

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# 固定されたインポート

import util # noqa
import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# 定数

g_vllm_port = 9999
g_vllm_model_name = "custom"

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "temperature=1.2"
)

# タイプ

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, skip_guesses: bool = False) -> str:
        if not self.empty:
            history = "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if not skip_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
            )
            return history
        return "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("無効な回答です、'yes'または'no'である必要があります")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}yes/no{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="質問").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="物/場所").to_pretty_xml()


# 関数

async def ask(base: rg.ChatPipeline, observation: Observation) -> str:
    if observation.step == 0:
        # キーワードバグが修正されるまで、最初の質問をオーバーライド
        return "私たちは20の質問をプレイしていますか？"
    
    
    full_question = f"""\
                あなたは現在次の質問をしています。

                <game-history>
                {observation.get_history_as_xml(skip_guesses=True)}
                </game-history>

                上記の履歴に基づき、次に最も有用なはい/いいえの質問をし、以下のフォーマットにしてください：
                {Question.xml_example()}

                - あなたのレスポンスは、最も多くの情報を集める焦点を絞った質問であるべきです
                - 質問は一般的なものから始めてください
                - 残りの探索空間を二分してください
                - 以前の質問と回答に注意してください

                あなたの次の質問は何ですか？
                """
    
    print(' ======質問中 ======')
    print(full_question)
    
    
    try:
        chat = await (
             base.fork(full_question)
            .until_parsed_as(Question, attempt_recovery=True, max_rounds=20)
            .run()
        )
        return chat.last.parse(Question).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return '人ですか？'

async def answer(base: rg.ChatPipeline, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("回答者にキーワードが提供されていません", file=sys.stderr)
        return "yes" # キーワードバグが修正されるまでオーバーライド
    
    last_question = observation.questions[-1]

    try:
        responses = []
        for i in range(5):
            # 5回ループし、最も頻繁なレスポンスを取得
            chat = await (
                base.fork(
#                     f"""\
#                         20の質問ゲーム。次のキーワードに対してはい/いいえで回答してください：[ {observation.keyword} ]

#                             質問: {last_question}

#                             ルール:
#                             1.  [{observation.keyword}] のみを考慮してください
#                             2. 各文字の質問のために文字を確認してください
#                             3. はいまたはいいえのみで答えてください

#                             フォーマット:
#                             <answer>yes</answer>
#                             または
#                             <answer>no</answer>

#                             あなたの回答は：
#                         """
                    f"""
                    キーワード: [ {observation.keyword} ]

                    Q: {last_question}

                    フォーマット: <answer>yes</answer> または <answer>no</answer>ではいまたはいいえで答えてください
                    """
                )
                .until_parsed_as(Answer, attempt_recovery=True, max_rounds=20)
                .run()
            )
            responses.append(chat.last.parse(Answer).content.strip('*'))
            
        print(f'レスポンスは {responses}')
        return pd.Series(responses).value_counts().index[0]
    except rg.error.MessagesExhaustedMaxRoundsError:
        print('%%%%%%%%%%%% エラーが発生したため、yesを回答します %%%%%%%%%%%% ')
        return 'yes'

async def guess(base: rg.ChatPipeline, observation: Observation) -> str:
    try:

        chat = await (
            base.fork(
                f"""\
                あなたは現在キーワードの情報に基づく推測を行っています。

                <game-history>
                {observation.get_history_as_xml()}
                </game-history>

                上記の歴史に基づき、キーワードの次に最適な推測を1つ生成し、以下のフォーマットにしてください：
                {Guess.xml_example()}

                - 上記の歴史に基づいて重複した推測を避けてください
                - 推測は特定の人物、場所、または物である必要があります

                あなたの推測は何ですか？
                """
            )
            .until_parsed_as(Guess, attempt_recovery=True, max_rounds=20)
            .run()
        )

        return chat.last.parse(Guess).content.strip('*')
    except rg.error.MessagesExhaustedMaxRoundsError:
        return 'フランス'
    
# vLLMとGenerator

try:
    vllm = util.run_and_wait_for_port([
        "python", "-m",
        "vllm.entrypoints.openai.api_server",
        "--enforce-eager",
        "--model", str(g_model_path),
        "--port", str(g_vllm_port),
        "--served-model-name", g_vllm_model_name,
        "--dtype=half"
    ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

    print("vLLMが起動しました")
except ValueError:
    print('vLLMはすでに実行中です')
    
    
generator = rg.get_generator(g_generator_id)

base =  generator.chat("""\
あなたは20の質問ゲームの優れたプレイヤーです。あなたは正確で、集中しており、構造的なアプローチをとります。あなたは有用な質問を作成し、質問に対する回答を行ったり、キーワードに関する推測を行ったりします。

""")

# エントリーポイント
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは文字 '{letters[0]}' で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次のいずれかの文字で始まりますか: {formatted_letters}?"

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

# シンプルな質問者
class SimpleQuestionerAgent():
    def __init__(self, keyword_df: pd.DataFrame):
        self.keyword_df = keyword_df
        self.keyword_df_init = keyword_df.copy()
        self.round = 0
        self.category_questions = [
            "私たちは20の質問をプレイしていますか？",
            "キーワードは場所ではないものですか？",
            "キーワードは場所ですか？",
        ]
        self.found_category = False
        
    def filter_keywords(self, obs):
        print(self.keyword_df.shape)
        # 以前の回答に基づいてkeyword_dfをフィルタリング
        for i, answer in enumerate(obs.answers):
            if obs.questions[i] in self.category_questions:
                if answer == 'yes':
                    if obs.questions[i] == "キーワードは場所ではないものですか？":
                        self.found_category = 'things'
                    if obs.questions[i] == "キーワードは場所ですか？":
                        self.found_category = 'place'
                    fc = self.found_category
                    self.keyword_df = self.keyword_df.query('category == @fc').reset_index(drop=True)
    
            if obs.questions[i].startswith('キーワードは次の文字で始まりますか'):
                if self.keyword_df['first_letter'].nunique() <= 1:
                    break
                letter_question = obs.questions[i]
                letters = extract_letters_from_question(letter_question)
                self.keyword_df = self.keyword_df.reset_index(drop=True).copy()
                if obs.answers[i] == 'yes':
                    self.keyword_df = self.keyword_df.loc[
                        self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
                elif obs.answers[i] == 'no':
                    self.keyword_df = self.keyword_df.loc[
                        ~self.keyword_df['first_letter'].isin(letters)].reset_index(drop=True).copy()
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
            
    def get_letters(self, obs, max_letters=20):
        n_letters = self.keyword_df['first_letter'].nunique()
        sample_letters = self.keyword_df['first_letter'].drop_duplicates().sample(n_letters // 2).values.tolist()
        sample_letters = sample_letters[:max_letters]
        print('サンプル文字', n_letters, sample_letters)
        return sample_letters # ', '.join(sample_letters)
    
    def __call__(self, obs, *args):
        if len(self.keyword_df) == 0:
            # リセット
            self.keyword_df = self.keyword_df_init.copy()
        self.filter_keywords(obs)
        if obs.turnType == 'ask':
            self.round += 1
            if (self.round <= 3 and not self.found_category):
                response = self.category_questions[self.round - 1]
            else:
                sample_letters = self.get_letters(obs)
                if len(sample_letters) == 0:
                    n_sample = min(len(self.keyword_df), 10)
                    possible_keywords = ", ".join(self.keyword_df['keyword'].sample(n_sample).values.tolist())
                    response = f"キーワードは次のいずれかのものでしょうか？ {possible_keywords}"
                else:
                    sample_letters_str = str(sample_letters).replace('[','').replace(']','')
                    response = format_first_letter_question(sample_letters)
        elif obs.turnType == 'guess':
            response = self.keyword_df['keyword'].sample(1).values[0]
            # 推測された単語を取り除く
            updated_df = self.keyword_df.loc[self.keyword_df['keyword'] != response].reset_index(drop=True).copy()
            if len(updated_df) >= 1:
                self.keyword_df = updated_df.copy()
            else:
                self.keyword_df = self.keyword_df_init.copy() # dfをリセット
#         print(f'ラウンド {self.round}')
#         print(f"{response=}")
#         print(f'keyword_dfサイズ {self.keyword_df.shape}')
        return response


keyword_df = pd.read_parquet(f'{g_agent_path}/keywords.parquet')
question_agent = None

async def observe(obs: t.Any) -> str:
    observation = Observation(**obs.__dict__)
    global question_agent
    if question_agent is None:
        question_agent = SimpleQuestionerAgent(keyword_df)

    try:
        match observation.turnType:
            case "ask":
                return question_agent(obs)
            case "answer":
                return await answer(base, observation)
            case "guess":
                return question_agent(obs)

            case _:
                raise ValueError("不明なターンタイプ")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

def agent_fn(obs: t.Any, _: t.Any) -> str:
    # 彼らのフレームワークで実行するときの非同期ゲート
    import asyncio
    return asyncio.run(observe(obs))

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Test the Agent Against Itself

</div>
<div class="column-right">

# 日本語訳

# エージェントを自己評価する


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def format_first_letter_question(letters):
    if not letters:
        return "Does the keyword start with any letter?"
    
    if len(letters) == 1:
        return f"Does the keyword start with the letter '{letters[0]}'"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" or '{letters[-1]}'"
    
    return f"Does the keyword start with one of the letters {formatted_letters}?"

format_first_letter_question(['a','b','c'])

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches
```

</div>
<div class="column-right">

# 日本語訳

```python
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは文字 '{letters[0]}' で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次のいずれかの文字で始まりますか: {formatted_letters}?"

format_first_letter_question(['a','b','c'])

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches
```

</div>
</details>

In [ ]:
def format_first_letter_question(letters):
    if not letters:
        return "キーワードはどの文字で始まりますか？"
    
    if len(letters) == 1:
        return f"キーワードは文字 '{letters[0]}' で始まりますか？"
    
    formatted_letters = ", ".join(f"'{letter}'" for letter in letters[:-1])
    formatted_letters += f" または '{letters[-1]}'"
    
    return f"キーワードは次のいずれかの文字で始まりますか: {formatted_letters}?"

format_first_letter_question(['a','b','c'])

import re

def extract_letters_from_question(question):
    pattern = r"'([a-zA-Z])'"
    matches = re.findall(pattern, question)
    return matches

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%load_ext autoreload
%autoreload 2
from main import Observation, agent_fn, observe
```

</div>
<div class="column-right">

# 日本語訳

```python
%load_ext autoreload
%autoreload 2
from main import Observation, agent_fn, observe
```

</div>
</details>

In [ ]:
%load_ext autoreload
%autoreload 2
from main import Observation, agent_fn, observe

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Check if vllm is running
!ps -aef | grep vllm
```

</div>
<div class="column-right">

# 日本語訳

```python
# vllmが実行中か確認
!ps -aef | grep vllm
```

</div>
</details>

In [ ]:
# vllmが実行中か確認
!ps -aef | grep vllm

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import pandas as pd

keyword_df = pd.read_parquet('keywords.parquet')
sample = keyword_df.sample(1)

obs = Observation(step = 0,
    role = 'guesser',
    turnType= "ask",
    keyword= sample['keyword'].values[0],
    category= sample['category'].values[0],
    questions = [],
    answers= [],
    guesses= [],
)

question_agent = None

for i in range(20):
    obs.role = 'guesser'
    obs.turnType = 'ask'
    question = await observe(obs)
    print(f'[{i} Question]: {question}')
    obs.questions.append(question)
    obs.role = 'answerer'
    obs.turnType = 'answer'
    answer = await observe(obs)
    obs.answers.append(answer)
    
    if obs.questions[-1].startswith('Are we playing 20 questions?'):
        gt_answer = answer # whatever
    elif obs.questions[-1].startswith('Is the keyword a thing that is not a location?'):
        if sample['category'].values[0] == 'things':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('Is the keyword a place?'):
        if sample['category'].values[0] == 'place':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('Does the keyword start'):
        letters_guess = extract_letters_from_question(obs.questions[-1])
        gt_answer = obs.keyword[0] in letters_guess
        gt_answer = 'yes' if gt_answer else 'no'
    elif obs.questions[-1].startswith('Is the keyword one of the following?'):
        possible_kw = obs.questions[-1].replace('Is the keyword one of the following? ','').split(',')
        possible_kw = [c.strip(' ') for c in possible_kw]
        print(possible_kw)
        gt_answer = obs.keyword in possible_kw
        gt_answer = 'yes' if gt_answer else 'no'

    print(f'[{i} Answer]: {answer} [True Answer]: {gt_answer}')
    if answer != gt_answer:
        break

    obs.role = 'guesser'
    obs.turnType = 'guess'
    guess = await observe(obs)
    print(f'[{i} Guess]: {guess} - [Keyword]: {obs.keyword}')
    obs.guesses.append(guess)
    if guess == obs.keyword:
        print('GOT IT!')
        break
        
    obs.step += 1
```

</div>
<div class="column-right">

# 日本語訳

```python
import pandas as pd

keyword_df = pd.read_parquet('keywords.parquet')
sample = keyword_df.sample(1)

obs = Observation(step = 0,
    role = 'guesser',
    turnType= "ask",
    keyword= sample['keyword'].values[0],
    category= sample['category'].values[0],
    questions = [],
    answers= [],
    guesses= [],
)

question_agent = None

for i in range(20):
    obs.role = 'guesser'
    obs.turnType = 'ask'
    question = await observe(obs)
    print(f'[{i} 質問]: {question}')
    obs.questions.append(question)
    obs.role = 'answerer'
    obs.turnType = 'answer'
    answer = await observe(obs)
    obs.answers.append(answer)
    
    if obs.questions[-1].startswith('私たちは20の質問をプレイしていますか？'):
        gt_answer = answer # 何でも
    elif obs.questions[-1].startswith('キーワードは場所ではないものですか？'):
        if sample['category'].values[0] == 'things':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('キーワードは場所ですか？'):
        if sample['category'].values[0] == 'place':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('キーワードは次の文字で始まりますか'):
        letters_guess = extract_letters_from_question(obs.questions[-1])
        gt_answer = obs.keyword[0] in letters_guess
        gt_answer = 'yes' if gt_answer else 'no'
    elif obs.questions[-1].startswith('キーワードは次のいずれかのものでしょうか？'):
        possible_kw = obs.questions[-1].replace('キーワードは次のいずれかのものでしょうか？ ','').split(',')
        possible_kw = [c.strip(' ') for c in possible_kw]
        print(possible_kw)
        gt_answer = obs.keyword in possible_kw
        gt_answer = 'yes' if gt_answer else 'no'

    print(f'[{i} 回答]: {answer} [真の回答]: {gt_answer}')
    if answer != gt_answer:
        break

    obs.role = 'guesser'
    obs.turnType = 'guess'
    guess = await observe(obs)
    print(f'[{i} 推測]: {guess} - [キーワード]: {obs.keyword}')
    obs.guesses.append(guess)
    if guess == obs.keyword:
        print('正解です！')
        break
        
    obs.step += 1
```

</div>
</details>

In [ ]:
import pandas as pd

keyword_df = pd.read_parquet('keywords.parquet')
sample = keyword_df.sample(1)

obs = Observation(step = 0,
    role = 'guesser',
    turnType= "ask",
    keyword= sample['keyword'].values[0],
    category= sample['category'].values[0],
    questions = [],
    answers= [],
    guesses= [],
)

question_agent = None

for i in range(20):
    obs.role = 'guesser'
    obs.turnType = 'ask'
    question = await observe(obs)
    print(f'[{i} 質問]: {question}')
    obs.questions.append(question)
    obs.role = 'answerer'
    obs.turnType = 'answer'
    answer = await observe(obs)
    obs.answers.append(answer)
    
    if obs.questions[-1].startswith('私たちは20の質問をプレイしていますか？'):
        gt_answer = answer # 何でも
    elif obs.questions[-1].startswith('キーワードは場所ではないものですか？'):
        if sample['category'].values[0] == 'things':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('キーワードは場所ですか？'):
        if sample['category'].values[0] == 'place':
            gt_answer = 'yes'
        else:
            gt_answer = 'no'
    elif obs.questions[-1].startswith('キーワードは次の文字で始まりますか'):
        letters_guess = extract_letters_from_question(obs.questions[-1])
        gt_answer = obs.keyword[0] in letters_guess
        gt_answer = 'yes' if gt_answer else 'no'
    elif obs.questions[-1].startswith('キーワードは次のいずれかのものでしょうか？'):
        possible_kw = obs.questions[-1].replace('キーワードは次のいずれかのものでしょうか？ ','').split(',')
        possible_kw = [c.strip(' ') for c in possible_kw]
        print(possible_kw)
        gt_answer = obs.keyword in possible_kw
        gt_answer = 'yes' if gt_answer else 'no'

    print(f'[{i} 回答]: {answer} [真の回答]: {gt_answer}')
    if answer != gt_answer:
        break

    obs.role = 'guesser'
    obs.turnType = 'guess'
    guess = await observe(obs)
    print(f'[{i} 推測]: {guess} - [キーワード]: {obs.keyword}')
    obs.guesses.append(guess)
    if guess == obs.keyword:
        print('正解です！')
        break
        
    obs.step += 1

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# last_question = "Does the keyword start with one of the letters 'a', 'n' or 'g'?"
# out = await generator.chat(f"""\
# You will percisely answer a yes/no question for the keyword [{obs.keyword}] in the category [{obs.category}]

# [QUESTION] "{last_question}" [/QUESTION]

# Remember they keyword is [{obs.keyword}] - The QUESTION should be answered percisely for this specific keyword.

# Answer the yes/no question above and place it in the following format:
# {Answer.xml_example()}
# """).run()
```

</div>
<div class="column-right">

# 日本語訳

```python
# last_question = "キーワードは次のいずれかの文字で始まりますか: 'a', 'n', または 'g'?"
# out = await generator.chat(f"""\
# あなたはキーワード[{obs.keyword}]に対してはい/いいえの質問に正確に答えます。カテゴリは[{obs.category}]です。

# [質問] "{last_question}" [/質問]

# キーワードは[{obs.keyword}]です - 質問にはこの特定のキーワードのために正確に答えてください。

# 上記のはい/いいえの質問に答え、以下のフォーマットにしてください：
# {Answer.xml_example()}
# """).run()
```

</div>
</details>

In [ ]:
# last_question = "キーワードは次のいずれかの文字で始まりますか: 'a', 'n', または 'g'?"
# out = await generator.chat(f"""\
# あなたはキーワード[{obs.keyword}]に対してはい/いいえの質問に正確に答えます。カテゴリは[{obs.category}]です。

# [質問] "{last_question}" [/質問]

# キーワードは[{obs.keyword}]です - 質問にはこの特定のキーワードのために正確に答えてください。

# 上記のはい/いいえの質問に答え、以下のフォーマットにしてください：
# {Answer.xml_example()}
# """).run()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

# last_question = "Does the keyword start with one of the letters 'a', 'n' or 'g'?"
# out = await generator.chat(f"""\
# You are playing a 20 questions game. Your task is to accurately answer a yes/no question about a specific keyword.

# Keyword: [{obs.keyword}]
# Category: [{obs.category}]

# Question: {last_question}

# Instructions:
# 1. Consider ONLY the keyword [{obs.keyword}] when answering.
# 2. Answer the question with either 'yes' or 'no'.
# 3. Double-check your answer for accuracy before responding.
# 4. Provide your answer in the XML format shown below.

# Your response should be in this exact format:
# <answer>yes</answer>
# OR
# <answer>no</answer>

# Now, please answer the question accurately for the keyword [{obs.keyword}]:
# """
#                           ).run()
```

</div>
<div class="column-right">

# 日本語訳

```python
# last_question = "キーワードは次のいずれかの文字で始まりますか: 'a', 'n', または 'g'?"
# out = await generator.chat(f"""\
# あなたは20の質問ゲームをプレイしています。あなたの任務は特定のキーワードについてはっきりとしたはい/いいえの質問に答えることです。

# キーワード: [{obs.keyword}]
# カテゴリ: [{obs.category}]

# 質問: {last_question}

# 指示:
# 1. 答える際にはキーワード[{obs.keyword}] のみを考慮してください。
# 2. はいまたはいいえで答えてください。
# 3. 答えを正確にするために再確認してください。
# 4. 下記のXML形式で回答を提供してください。

# あなたの応答はこの正確な形式であるべきです:
# <answer>yes</answer>
# または
# <answer>no</answer>

# さあ、キーワード[{obs.keyword}]に対して質問に正確に答えてください:
# """
#                           ).run()
```

</div>
</details>

In [ ]:
# last_question = "キーワードは次のいずれかの文字で始まりますか: 'a', 'n', または 'g'?"
# out = await generator.chat(f"""\
# あなたは20の質問ゲームをプレイしています。あなたの任務は特定のキーワードについてはっきりとしたはい/いいえの質問に答えることです。

# キーワード: [{obs.keyword}]
# カテゴリ: [{obs.category}]

# 質問: {last_question}

# 指示:
# 1. 答える際にはキーワード[{obs.keyword}] のみを考慮してください。
# 2. はいまたはいいえで答えてください。
# 3. 答えを正確にするために再確認してください。
# 4. 下記のXML形式で回答を提供してください。

# あなたの応答はこの正確な形式であるべきです:
# <answer>yes</answer>
# または
# <answer>no</answer>

# さあ、キーワード[{obs.keyword}]に対して質問に正確に答えてください:
# """
#                           ).run()

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# print(out.prev[-1].content)
```

</div>
<div class="column-right">

# 日本語訳

```python
# print(out.prev[-1].content)
```

</div>
</details>

In [ ]:
# print(out.prev[-1].content)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# out.last
```

</div>
<div class="column-right">

# 日本語訳

```python
# out.last
```

</div>
</details>

In [ ]:
# out.last

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Zipping Model and Code for Submission

</div>
<div class="column-right">

# 日本語訳

# モデルとコードを提出用に圧縮


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!apt install pigz pv
```

</div>
<div class="column-right">

# 日本語訳

```python
!apt install pigz pv
```

</div>
</details>

In [ ]:
!apt install pigz pv

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \
    -C /kaggle/working main.py util.py \
    -C /kaggle/working keywords.parquet
```

</div>
<div class="column-right">

# 日本語訳

```python
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \
    -C /kaggle/working main.py util.py \
    -C /kaggle/working keywords.parquet
```

</div>
</details>

In [ ]:
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \
    -C /kaggle/working main.py util.py \
    -C /kaggle/working keywords.parquet

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!ls -GFlash --color
```

</div>
<div class="column-right">

# 日本語訳

```python
!ls -GFlash --color
```

</div>
</details>

In [ ]:
!ls -GFlash --color

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

# Submitting using Kaggle CLI

Optionally you can submit using the kaggle cli interface without needing to re-run commit the notebook.

</div>
<div class="column-right">

# 日本語訳

# Kaggle CLIを使用して提出

必要に応じて、ノートブックを再実行せずにKaggle CLIインターフェースを使用して提出できます。


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# !KAGGLE_USERNAME={KAGGLE_USERNAME} \
#  KAGGLE_KEY={KAGGLE_KEY} \
#  kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "submit from notebook"
```

</div>
<div class="column-right">

# 日本語訳

```python
# !KAGGLE_USERNAME={KAGGLE_USERNAME} \
#  KAGGLE_KEY={KAGGLE_KEY} \
#  kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "ノートブックから提出"
```

</div>
</details>

In [ ]:
# !KAGGLE_USERNAME={KAGGLE_USERNAME} \
#  KAGGLE_KEY={KAGGLE_KEY} \
#  kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "ノートブックから提出"